## CNN 对MNIST数据集进行训练

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)

batch_size = 100

n_batch = mnist.train.num_examples // batch_size

# 参数摘要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean) # mean value
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev', stddev) # standard  deviation
        tf.summary.scalar('max', tf.reduce_max(var)) # max value
        tf.summary.scalar('min', tf.reduce_min(var)) # min value
        tf.summary.histogram('histogram', var) # histogram
        
# initialize weight
def weight_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1) # 生成一个截断的正态分布
    return tf.Variable(initial, name=name)

# initialize bias
def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

# convolution layer
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

# pooling layer
def max_pool_2x2(x):
    # kernel size [1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')

# name space
with tf.name_scope('input'):
    # define two placeholder
    x = tf.placeholder(tf.float32, [None,784], name='x-input')
    y = tf.placeholder(tf.float32, [None,10], name='y-input')
    with tf.name_scope('x_image'):
        x_image = tf.reshape(x,[-1,28,28,1], name='x_image')
        
with tf.name_scope('Conv1'):
    # initialize first conv layer
    with tf.name_scope('W_conv1'):
        W_conv1 = weight_variable([5,5,1,32], name= 'W_conv1') # 5*5 sampling windows  32 kernels
    with tf.name_scope('b_conv1'):
        b_conv1 = bias_variable([32], name='b_conv1')
    
    with tf.name_scope('conv2d_1'):
        conv2d_1 = conv2d(x_image, W_conv1) + b_conv1
    with tf.name_scope('relu'):
        h_conv1 = tf.nn.relu(conv2d_1)
    with tf.name_scope('h_pool1'):
        h_pool1 = max_pool_2x2(h_conv1)

with tf.name_scope('Conv2'):
    # initial second conv layer
    with tf.name_scope('W_conv2'):
        W_conv2 = weight_variable([5,5,32,64], name = 'W_conv2') # 5*5 sampling windows 64 kernels
    with tf.name_scope('b_conv2'):
        b_conv2 = bias_variable([64],name = 'b_conv2')
    # h_pool1 与 权值向量进行卷积， 加上偏置值 ，然后应用relu激活函数
    with tf.name_scope('conv2d_2'):
        conv2d_2 = conv2d(h_pool1, W_conv2) + b_conv2
    with tf.name_scope('h_pool2'):
        h_pool2 = max_pool_2x2(conv2d_2) # max pooling
        
with tf.name_scope('fc_1'):
    with tf.name_scope('W_fc1'):
        W_fc1 = weight_variable([7*7*64, 1024], name='W_fc1') # 上一层有7*7*64 神经元, 此层有1024个神经元
    with tf.name_scope('b_fc1'):
        b_fc1 = bias_variable([1024], name= 'b_fc1')
        
        
    with tf.name_scope('h_pool2_flat'):
        h_pool2_flat = tf.reshape(h_pool2, [-1,7*7*64], name = 'h_pool2_flat')
    
    with tf.name_scope('wx_plus_b1'):
        wx_plus_b1 = tf.matmul(h_pool2_flat, W_fc1) + b_fc1
    with tf.name_scope('relu'):
        h_fc1 = tf.nn.relu(wx_plus_b1)
        
    # keep prob means nn output probilities
    with tf.name_scope('keep_prob'):
        keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    with tf.name_scope('h_fc1_drop'):
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob, name='h_fc1_drop')

with tf.name_scope('fc_2'):
    with tf.name_scope('W_fc2'):
        W_fc2 = weight_variable([1024, 10], name='W_fc2') # 上一层有7*7*64 神经元, 此层有1024个神经元
    with tf.name_scope('b_fc1'):
        b_fc2 = bias_variable([10], name= 'b_fc2')
        
    with tf.name_scope('wx_plus_b2'):
        wx_plus_b2 = tf.matmul(h_fc1, W_fc2) + b_fc2
        
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b2)

with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction), name='cross_entropy')
    tf.summary.scalar('cross_entropy', cross_entropy)
    
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.math.argmax(prediction,1), tf.math.argmax(y,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)
        
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('logs/train', sess.graph)
    test_writer = tf.summary.FileWriter('logs/test', sess.graph)
    for i in range(1001):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(train_step, feed_dict = {x:batch_xs, y:batch_ys, keep_prob:0.5})
        
        # record train set parameters
        summary = sess.run(merged, feed_dict = {x:batch_xs, y:batch_ys, keep_prob:1.0})
        train_writer.add_summary(summary, i)
        
        # record test set parameters
        batch_xs, batch_ys = mnist.test.next_batch(batch_size)
        summary = sess.run(merged, feed_dict={x:batch_xs,y:batch_ys, keep_prob:1.0})
        test_writer.add_summary(summary, i)
        
        if i %100 ==0:
            test_acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
            train_acc = sess.run(accuracy, feed_dict={x:mnist.train.images[:10000], y:mnist.train.labels[:10000], keep_prob:1.0})
            print('Iter ' + str(i) + ', Testing Acc = '+ str(test_acc) + ', Trainning Acc= '+ str(train_acc))
        
        

W0829 22:40:35.041215 25648 deprecation.py:323] From <ipython-input-2-74365de110cc>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0829 22:40:35.044217 25648 deprecation.py:323] From H:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0829 22:40:35.045215 25648 deprecation.py:323] From H:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implemen

Extracting MNIST_data\train-images-idx3-ubyte.gz


W0829 22:40:35.280139 25648 deprecation.py:323] From H:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0829 22:40:35.287137 25648 deprecation.py:323] From H:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0829 22:40:35.340120 25648 deprecation.py:323] From H:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Plea

Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


W0829 22:40:35.598037 25648 deprecation.py:506] From <ipython-input-2-74365de110cc>:91: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0829 22:40:35.618032 25648 deprecation.py:323] From <ipython-input-2-74365de110cc>:106: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Iter 0, Testing Acc = 0.0698, Trainning Acc= 0.0849
Iter 100, Testing Acc = 0.5506, Trainning Acc= 0.558
Iter 200, Testing Acc = 0.7559, Trainning Acc= 0.7579
Iter 300, Testing Acc = 0.7659, Trainning Acc= 0.7659
Iter 400, Testing Acc = 0.8575, Trainning Acc= 0.8517
Iter 500, Testing Acc = 0.8607, Trainning Acc= 0.8555
Iter 600, Testing Acc = 0.8672, Trainning Acc= 0.8721
Iter 700, Testing Acc = 0.8687, Trainning Acc= 0.8756
Iter 800, Testing Acc = 0.8708, Trainning Acc= 0.8776
Iter 900, Testing Acc = 0.8719, Trainning Acc= 0.8759
Iter 1000, Testing Acc = 0.8777, Trainning Acc= 0.8804
